In [ ]:
import pathlib
import lzma
import re
import os
import datetime
import copy

import numpy as np
import pandas as pd

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys._utilities.filesystem

In [ ]:
ip = '192.168.100.200'
root = pathlib.Path(r'\\physics-server\iComLogFiles')
live_path = root.joinpath(f'live/{ip}.txt')

In [ ]:
offset = 20000
date_pattern = re.compile(b'\d\d\d\d-\d\d-\d\d\d\d:\d\d:\d\d.')

In [ ]:



# def pull_header(tag, length, results_dict, results):
#     index = np.where(results == tag)[0]

#     for i, ref in enumerate(index):
#         asymx = results[ref+1:ref+length+1]
#         asymx = np.array(asymx).astype(float)
#         asymx[(asymx == -32767) | (asymx == 32767)] = None
#         results_dict["{}-{}".format(tag, i)] = asymx

#     for ref in index[-1::-1]:
#         results = np.delete(results, np.arange(ref,ref+length+1))
        
#     return results_dict, results


# def organise_by_tags(results):
#     results_dict = dict()

#     results_dict, results = pull_header('ASYMX', 2, results_dict, results)
#     results_dict, results = pull_header('ASYMY', 2, results_dict, results)
#     results_dict, results = pull_header('MLCX', 160, results_dict, results)

#     pattern = re.compile('[a-zA-Z][a-zA-Z0-9 -]+')

#     alpha_numeric = np.array([
#         pattern.match(value)
#         for value in results
#     ]).astype(bool)
#     results_dict["Text Tags"] = results[alpha_numeric].tolist()
#     results = np.delete(results, np.where(alpha_numeric)[0])

#     left_overs = results.astype(str)
#     left_overs[(left_overs == '-32767') | (left_overs == '32767')] = None
    
#     left_overs = left_overs.tolist()
    
#     results_dict["Patient ID"] = left_overs.pop(0)
#     results_dict["Segment"] = left_overs.pop(0)
#     assert results_dict["Segment"] == left_overs.pop(0)
#     assert results_dict["Segment"] == left_overs.pop(0)
#     results_dict["Monitor Units"] = left_overs.pop(0)

#     results_dict["Left Overs"] = left_overs
    
#     return results_dict


# def convert(data_point):
#     results = initial_results_parse(data_point)
#     results_dict = organise_by_tags(results)
    
#     return results_dict

In [ ]:
# def update(data=None):
#     if data is None:
#         with pymedphys._utilities.filesystem.open_no_lock(live_path, 'rb') as f:
#             f.seek(0, os.SEEK_END)
#             file_size = f.tell()
#             f.seek(file_size - offset)
#             data = f.read()
        
#     date_index = [m.span() for m in pattern.finditer(data)]
#     start_points = [
#         span[0] - 8 for span in date_index
#     ]

#     end_points = start_points[1::] + [None]

#     data_points = [data[start:end] for start, end in zip(start_points, end_points)]
    
#     return convert(data_points[-1])


# update()

In [ ]:
# update(data)

In [ ]:
def get_most_recent_data_point():
    with pymedphys._utilities.filesystem.open_no_lock(live_path, 'rb') as f:
        f.seek(0, os.SEEK_END)
        file_size = f.tell()
        f.seek(file_size - offset)
        data = f.read()
        
    date_index = [m.span() for m in date_pattern.finditer(data)]
    start_points = [
        span[0] - 8 for span in date_index
    ]

    end_points = start_points[1::] + [None]

    data_points = [data[start:end] for start, end in zip(start_points, end_points)]
    return data_points[-1]

In [ ]:
# data = get_most_recent_data_point()
# new_lines = data.split(b'\n')
# new_lines

In [ ]:
# current_data

In [ ]:
def extract_positions_by_header(key, num_items, results_by_line):
    header_index = results_by_line.index([key])
    return [float(item[0]) for item in results_by_line[header_index + 1:header_index + 1 + num_items]]


def get_jaw_and_mlc(results_by_line):
    options = [
        (b'ASYMX', 2), (b'ASYMY', 2), (b'MLCX', 160)
    ]
    collimation = {}
    for key, num_items in options:
        collimation[key] = extract_positions_by_header(key, num_items, results_by_line)
        
    return collimation

In [ ]:
item_pattern = re.compile(b'\x00\x00\x00([a-zA-Z0-9 \.-]+)')

In [ ]:
# 0\xb8\x00DS\x00R\x04\x00\x00\x00MLCX
# \n0\x1c\x01DS\x00R\x06\x00\x00\x00-10.00\n0\x1c\x01DS\x00R\x06\x00\x00\x00-10.00\n0\x1

def extract_coll(data, label, number):
    header = b'0\xb8\x00DS\x00R[\x03\x04\x05\x06]\x00\x00\x00' + label + b'\n'
    item = b'0\x1c\x01DS\x00R[\x03\x04\x05\x06]\x00\x00\x00(-?\d+\.\d+)'

    regex = re.compile(header + b'\n'.join([item,]*number))

    match = regex.search(data)
    span = match.span()

    data = data[0:span[0]] + data[span[1]+1::]
    items = np.array([float(item) for item in match.groups()])
    
    return data, items

In [ ]:
# current_data = copy.copy(data)

In [ ]:
#0\x07\x10DS\x00P\x01\x00\x00\x001\x010\x07\x10DS\x00S\x01\x00\x00\x001\x010\x07\x10DS\x00R\x01\x00\x00\x001\x010\t


def extract_by_lookup(data, key):
    regex = re.compile(b'[0\x00pP]' + key + b'\x00[PSR][\x01\x02\x03\x04\x05\x06]\x00\x00\x00([a-zA-Z0-9 \.-]+)')
    match = regex.search(data)

    span = match.span()
    removed_data = data[0:span[0]] + data[span[1]+1::]
    
    return removed_data, match.group(1)

def extract_all_lookup(data, result):
    for key, label in lookup.items():
        try:
            data, result[label] = extract_by_lookup(data, key)
        except AttributeError:
            result[label] = None
        
    return data

def run_all_assertions(data, result):
    for key, label in assert_equal.items():
        try:
            while True:
                data, new_result = extract_by_lookup(data, key)
                if result[label] != new_result:
                    raise ValueError(f"Expected these to be the same:\n{label}\n{result[label]}\n{new_result}")
        except AttributeError:
            pass  
        
    return data
        

def strict_extract(data):
    result = dict()
    
    data = extract_all_lookup(data, result)
    
    data, result['mlc'] = extract_coll(data, b'MLCX', 160)
    data, result['jaw_x'] = extract_coll(data, b'ASYMX', 2)
    data, result['jaw_y'] = extract_coll(data, b'ASYMY', 2)
    
    data = run_all_assertions(data, result)
    
    return data, result

lookup = {
    b' \x00LO': 'Patient ID',
    b'\xb2\x00SH': 'Machine ID',
    b'\xc6\x00CS': 'Radiation Type',
    b'\x14\x01SH': 'Energy',
    b'\x18\x01CS': 'Wedge',
    b'\x07\x10DS': 'Segment',
    b'\t\x10DS': 'Total MU',
    b'2\x00DS': 'Delivery MU',
    b'3\x00DS': 'Backup Delivery MU',
    b'8\x00SH': 'Beam Timer',
    b'\x0b\x00DS': 'Segment MU',
    b'\x1e\x01DS': 'Gantry',
    b' \x01DS': 'Collimator',
    b'\"\x01DS' : 'Table Column',
    b'\%\x01DS' : 'Table Isocentric',
    b'\(\x01DS' : 'Table Vertical',
    b'\)\x01DS' : 'Table Longitudinal',
    b'\*\x01DS' : 'Table Lateral',
}

assert_equal = {
    b'\x07\x10DS': 'Segment',
    b'\x06\x10LO': 'Machine ID',
}

data = get_most_recent_data_point()
shrunk_data, result = strict_extract(data)

print(result)

def initial_results_parse(data_point):
    pattern = re.compile(b'(........\x00\x00\x00[a-zA-Z0-9 \.-]+)')

    results = pattern.findall(data_point)
    results = np.array(results)
    
    return results

initial_results_parse(shrunk_data)

In [ ]:
# # data = get_most_recent_data_point()
# new_lines = data.split(b'\n')
# results_by_line = [item_pattern.findall(line) for line in new_lines]
# results = dict()

# results['Patient ID'] = results_by_line[0][0].decode()
# results['Segment'] = int(results_by_line[0][1])
# assert results['Segment'] == int(results_by_line[0][2])
# assert results['Segment'] == int(results_by_line[0][3])
# results['Delivery MU'] = float(results_by_line[0][4])
# # assert results['Delivery MU'] == float(results_by_line[1][-2])
# results['Backup Delivery MU'] = float(results_by_line[1][-1])
# results['Dose Rate'] = float(results_by_line[1][-3])

# results['Segment MU'] = float(results_by_line[-1][-1])
# results['Beam Timer'] = float(results_by_line[-1][5])

# results['Machine ID'] = results_by_line[2][0].decode()
# results['Radiation Type'] = results_by_line[3][0].decode()
# results['Energy'] = results_by_line[7][0].decode()
# results['Wedge'] = results_by_line[9][0].decode()

# results['Gantry'] = float(results_by_line[-8][0])
# results['Collimator'] = float(results_by_line[-7][0])

# results['Table Column'] = int(results_by_line[-6][0])
# results['Table Isocentric'] = int(results_by_line[-5][0])

# results['Table Vertical'] = float(results_by_line[-4][0])
# results['Table Longitudinal'] = float(results_by_line[-3][0])
# results['Table Lateral'] = float(results_by_line[-2][0])

# collimation = get_jaw_and_mlc(results_by_line)

# mlcs = collimation[b'MLCX']
# mlc_a = mlcs[0::2]
# mlc_b = mlcs[1::2]

# results['MLC-A'] = mlc_a
# results['MLC-B'] = mlc_b

# results['JAW'] = collimation[b'ASYMY']



# print(results.keys())

# results_by_line

In [ ]:
# num_items = 2
# header_index = results_by_line.index([b'ASYMX'])
# [float(item[0]) for item in results_by_line[header_index + 1:header_index + 1 + num_items]]